In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=v2CokX8bvExwyj6O5oGnmeVrxPmBsAYvbvjKjtR6Fmk&tc=AMOIQukU-QCCGcJl_OM_Owb4makdqf7nN5p4vVk6R8c&cc=r-W-0kkUrNPWPUc_8m1lfu88XD0bGOvKecUWK_jmT4Y

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnbTG72IOuN5l1BbMdRkVFNp7Xry3BC_eeZrIAQoW5a5LzhVkV68vQ

Successfully saved authorization token.


In [3]:
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
India = countries.filter(ee.Filter.eq('country_na', 'India'))

In [4]:
# Input imagery is a cloud free Landsat 8 composite
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1")

# Define a smaller region of interest (ROI)
roi = ee.Geometry.Rectangle([77, 12, 78.5, 13.5])
l8 = l8.filterBounds(roi)

image = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': l8.filterDate('2018-01-01', '2018-01-31'),
     'asFloat': True
})

In [5]:
# Use these bands for prediction
bands = ['B1', 'B2', 'B3', 'B4', 'B5','B6','B7', 'B10', 'B11']

In [6]:


forest1 = ee.Geometry.Polygon ([
    [78, 13],
    [78.2, 13],
    [78.2, 13.2],
    [78, 13.2],
    [78, 13]
])
forest2 = ee.Geometry.Polygon ([
    [78, 13],
    [78.2, 13],
    [78.2, 13.2],
    [78, 13.2],
    [78, 13]
])

nonforest1 =ee.Geometry.Polygon([
    [72.5, 18.5],
    [72.7, 18.5],
    [72.7, 18.7],
    [72.5, 18.7],
    [72.5, 18.5]
])
nonforest2 =ee.Geometry.Polygon([
    [72.5, 18.5],
    [72.7, 18.5],
    [72.7, 18.7],
    [72.5, 18.7],
    [72.5, 18.5]
])

In [7]:
# Make a feature collection from the hand-made geometries
polygons = ee.FeatureCollection([
  ee.Feature(nonforest1, {'class': 0}),
  ee.Feature(nonforest2, {'class': 0}),
  ee.Feature(forest1, {'class': 1}),
  ee.Feature(forest2, {'class': 1}),
])

In [8]:
#Get the values for all the pixels in each polygon in the training
training = image.sampleRegions(**{
    # Get the sample form the polygons FeatureCollection
    'collection': polygons,
    # Keep this list of properties from the polygons
     'properties': ['class'],
      #Set the scale to get Landsat pixels in the polygons
      'scale': 60
})

In [9]:
# Create an SVM classifier with custom parameters
classifier = ee.Classifier.libsvm(**{
    'kernelType': 'RBF',
    'gamma': 0.5,
    'cost': 10
})

In [10]:
# Train the classifier
trained = classifier.train(training, 'class', bands)


In [11]:
# Classify the image
classified = image.classify(trained)

# Clip image
clip = classified.clip(India)

In [12]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[12.75, 77.75], zoom_start=10)

# Add the layer to the map object.
my_map.add_ee_layer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.5, 'gamma': 2}, "image")
my_map.add_ee_layer(classified, {'min': 0, 'max': 1, 'palette': ['red', 'green']}, 'deforestation')


# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)